In [258]:
!pip install selenium

In [259]:
!pip install undetected-chromedriver

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc

In [6]:
driver = uc.Chrome("") #경로 생략, 자동설정
#코랩에서는 오류 발생
#주피터 노트북 사용해야함

In [8]:
driver.get("https://finance.naver.com/item/sise_day.naver?code=005930")

In [11]:
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

#driver.find_element(By.CSS_SELETOR, "table.type2")
html = BeautifulSoup(driver.page_source)

table = html.find("table", class_ = "type2")
pd.read_html(str(table))[0].dropna()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2024.10.07,59900.0,하락 700,60200.0,60500.0,59500.0,8179764.0
2,2024.10.04,60600.0,하락 700,61000.0,61700.0,60500.0,24247578.0
3,2024.10.02,61300.0,하락 200,60500.0,61900.0,59900.0,28473536.0
4,2024.09.30,61500.0,"하락 2,700",64200.0,64300.0,61500.0,32694163.0
5,2024.09.27,64200.0,하락 500,64700.0,65400.0,64200.0,28433030.0
9,2024.09.26,64700.0,"상승 2,500",63900.0,64900.0,63700.0,37566015.0
10,2024.09.25,62200.0,"하락 1,000",63800.0,64200.0,62200.0,28652438.0
11,2024.09.24,63200.0,상승 600,62800.0,63400.0,62400.0,26957500.0
12,2024.09.23,62600.0,하락 400,62300.0,63500.0,62200.0,28542376.0
13,2024.09.20,63000.0,하락 100,63800.0,64700.0,63000.0,32746056.0


### 셀레니움 이용해서 1~10페이지까지만 하나의 데이터프레임으로 만들기

In [20]:
import time

empty = []

for page in range(1, 10): 
    url = f"https://finance.naver.com/item/sise_day.naver?code=005930&page={page}"
    driver.get(url)  
    time.sleep(2)

    html = BeautifulSoup(driver.page_source)

    table = html.find("table", class_="type2")

    if table:
        df = pd.read_html(str(table))[0].dropna() 
        empty.append(df)

data = pd.concat(empty, ignore_index=True)

In [24]:
data.head(15)

,날짜,종가,전일비,시가,고가,저가,거래량
0,2024.10.07,60000.0,하락 600,60200.0,60500.0,59500.0,9034657.0
1,2024.10.04,60600.0,하락 700,61000.0,61700.0,60500.0,24247578.0
2,2024.10.02,61300.0,하락 200,60500.0,61900.0,59900.0,28473536.0
3,2024.09.30,61500.0,"하락 2,700",64200.0,64300.0,61500.0,32694163.0
4,2024.09.27,64200.0,하락 500,64700.0,65400.0,64200.0,28433030.0
5,2024.09.26,64700.0,"상승 2,500",63900.0,64900.0,63700.0,37566015.0
6,2024.09.25,62200.0,"하락 1,000",63800.0,64200.0,62200.0,28652438.0
7,2024.09.24,63200.0,상승 600,62800.0,63400.0,62400.0,26957500.0
8,2024.09.23,62600.0,하락 400,62300.0,63500.0,62200.0,28542376.0
9,2024.09.20,63000.0,하락 100,63800.0,64700.0,63000.0,32746056.0


### 끝페이지까지 다 들고 오기
강사님 풀이

In [29]:
box = []
for page in range(1, 710): 
    driver.get( f"https://finance.naver.com/item/sise_day.naver?code=005930&page={page}")
    driver.implicitly_wait(10)#페이지를 잘 못 들고오는 걸 방지하는 사기 함수 = 최대 10초 기다려라
    #시간 많으면 TIME쓰는게 나음
    
    html = BeautifulSoup(driver.page_source)
    table = html.find("table", class_="type2")
    table = pd.read_html(str(table))[0].dropna()
    box.append(table)

In [30]:
pd.concat(box)

,날짜,종가,전일비,시가,고가,저가,거래량
1,2024.10.07,59800.0,하락 800,60200.0,60500.0,59500.0,9572120.0
2,2024.10.04,60600.0,하락 700,61000.0,61700.0,60500.0,24247578.0
3,2024.10.02,61300.0,하락 200,60500.0,61900.0,59900.0,28473536.0
4,2024.09.30,61500.0,"하락 2,700",64200.0,64300.0,61500.0,32694163.0
5,2024.09.27,64200.0,하락 500,64700.0,65400.0,64200.0,28433030.0
...,...,...,...,...,...,...,...
3,1996.06.29,68500.0,상승 400,68100.0,69100.0,67100.0,96710.0
4,1996.06.28,68100.0,"상승 1,200",67300.0,68500.0,67200.0,138430.0
5,1996.06.27,66900.0,하락 800,67500.0,67700.0,66700.0,155450.0
9,1996.06.26,67700.0,상승 200,67600.0,67900.0,66000.0,136630.0


In [33]:
driver.page_source #실시간으로 보고있는 페이지 보여줌

'<html lang="ko"><head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버페이 증권</title>\n\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20240926170906/css/newstock.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20240926170906/css/common.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20240926170906/css/finance_header.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20240926170906/css/layout.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20240926170906/css/main.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20240926170906/css/newstock2.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20240926170906/css/newstock3.css">\n<link rel="styles

## 셀레니움으로 naver를 열었을 때 로그인이 되어있게 하기

In [174]:
driver = uc.Chrome()
driver.get("https://nid.naver.com/nidlogin.login?mode=form&url=https://www.naver.com/")

In [49]:
driver.find_element(By.CSS_SELECTOR, "input.input_id").send_keys("my id") #send_keys를 통해서 값이 기입됨

In [51]:
#시도)아이디, 비밀번호 치고 로그인하게하기 
driver.find_element(By.CSS_SELECTOR, "input.input_id").send_keys("kelly6161")
driver.find_element(By.CSS_SELECTOR, "input.input_pw").send_keys("@Imjiin61")

driver.find_element(By.CSS_SELECTOR, "button.btn_login").click()

##일반적인 로그인 방식이 아니여서 자동입력방지문자를 기입하라고함

강사님 해설

In [54]:
!pip install pyautogui

     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.2 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 653.8 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/171.2 kB ? eta -:--:--
     ------------------------------

In [92]:
import pyautogui

login_id = pyautogui.position() #원하는 위치에서 해당 코드를 돌리면 위치가 거기로 저장됨

In [110]:
pyautogui.click(login_id) #id칸을 클릭
time.sleep(1)
pyautogui.typewrite("kelly6161") #아이디 기입

In [118]:
login_pw = pyautogui.position()

In [120]:
pyautogui.click(login_pw)
time.sleep(1)
pyautogui.typewrite("@Imjiindlek61") 

In [122]:
driver.find_element(By.CSS_SELECTOR, "button.btn_login").click()

또 다른 방법 \
: 컨v 컨c 이용 \
: send key아니면 잘 안걸림

In [ ]:
import pyperclip

pyautogui.click(login_id)
time.sleep(1)
pyautogui.click(login_id)
time.sleep(1) #확실하게 두번 클릭

pyperclip.copy("kelly6161")
pyautogui.hotkey("ctrl","V")

pyautogui.click(login_pw)
time.sleep(1)
pyautogui.click(login_pw)
time.sleep(1)

pyperclip.copy("@Imjiindlek61")
pyautogui.hotkey("ctrl","V")

pyautogui.click(gg)

## 스크롤 결과 계속 로드하기

In [188]:
driver = uc.Chrome()
driver.get("https://shopping.naver.com/fresh/localfood/home")

In [ ]:
#얼만큼 내려가야하지?
while True:
    body = driver.find_element(By.CSS_SELECTOR, "body")
    body.send_keys(Keys.PAGE_DOWN) #페이지다운 버튼 누르기
    time.sleep(2)

In [194]:
#100개 이상의 상품 출력
#상품명, 가격, 평점 출력
#반복 출력 금지

total = set() #중복방지를 위해 집합 사용

while True: 
    for i in driver.find_elements(By.CSS_SELECTOR, "li._1OKV3UfxuI"): #여러개 찾을거라서 elements
        title = (i.find_element(By.CSS_SELECTOR, "strong._1lxNNH5gfD").text) #상품당 상품명은 하나
        price = (i.find_element(By.CSS_SELECTOR, "span._3MzrqSUlwR").text) #가격
        star = (i.find_element(By.CSS_SELECTOR, "span._29gNnX1yLx").text.split("\n")[1]) #가격
        a = (title, price, star)
        total.add(a)

    print(f"{len(total)}개 아이템 수집 완료")

    if len(total) >= 100:
        break

10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이템 수집 완료
10개 아이

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.71)
Stacktrace:
	GetHandleVerifier [0x011D7143+25587]
	(No symbol) [0x0116A2E4]
	(No symbol) [0x01062113]
	(No symbol) [0x0103E23B]
	(No symbol) [0x010D179F]
	(No symbol) [0x010E4CB9]
	(No symbol) [0x010CA936]
	(No symbol) [0x0109BA73]
	(No symbol) [0x0109C4CD]
	GetHandleVerifier [0x014B4C63+3030803]
	GetHandleVerifier [0x01506B99+3366473]
	GetHandleVerifier [0x012695F2+624802]
	GetHandleVerifier [0x01270E6C+655644]
	(No symbol) [0x01172C9D]
	(No symbol) [0x0116FD68]
	(No symbol) [0x0116FF05]
	(No symbol) [0x01162336]
	BaseThreadInitThunk [0x76507BA9+25]
	RtlInitializeExceptionChain [0x77B1C11B+107]
	RtlClearBits [0x77B1C09F+191]


In [197]:
a = list(total)
print(pd.DataFrame(a,columns = ['제목', '가격', '평점']))

                                               제목       가격    평점
0     쟁여두고 먹는 육즙 가득 떡갈비 너비아니 함박 스테이크 120g 5종 골라담기   1,990원  4.78
1                        여수엄마 간장게장 양념게장 알배기 간장꽃게장  29,900원  4.87
2                          집에서 즐기는 따끈따끈 돼지국밥 고기국밥   7,900원  4.88
3                        말바우시장선김치 국산 전라도 파 김치 1kg  21,000원  4.81
4          62년전통 알품게 삼대인천게장 간장게장 양념게장 연평도 꽃게장 암꽃게  38,000원  4.77
5      서민갑부 송쭈집 100억매출 양념 쭈꾸미 볶음 500g 300g 캠핑 밀키트   9,500원  4.85
6                          섬진강재첩체험마을 조개섬 재첩국 500g  20,000원  4.71
7           100% 통밀빵 520g 우리밀 무설탕 무달걀 비건 식사대용 통곡물   8,900원  4.83
8        [네이버명예의전당] 남원 현식당 추어탕 600g (1팩 당 7,000원)  35,000원  4.76
9  서민갑부 우정옥 도가니탕 1kg(2인분) 건더기 많고 쫀득한 스지 몸보신 사골 곰탕  17,900원  4.83


## 블로그 가져오기
crtl + shift + i : 소스코드 확인 가능

In [209]:
driver = uc.Chrome()
driver.get("https://blog.naver.com/stomato2202/223609540178")

In [205]:
print(i.find_element(By.CSS_SELECTOR, "div.pcol1")) #레이어가 많이 되어있어서 안됨

##기본 창과 메인프레임창 두 종류가 있음
# iframe이라는 소스코드가 있으면 창이 두 종류라는것
# MAIN을 바꿔야 필요한걸 긁어올 수 있음

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.71)
Stacktrace:
	GetHandleVerifier [0x011D7143+25587]
	(No symbol) [0x0116A2E4]
	(No symbol) [0x01062113]
	(No symbol) [0x0103E23B]
	(No symbol) [0x010D179F]
	(No symbol) [0x010E4CB9]
	(No symbol) [0x010CA936]
	(No symbol) [0x0109BA73]
	(No symbol) [0x0109C4CD]
	GetHandleVerifier [0x014B4C63+3030803]
	GetHandleVerifier [0x01506B99+3366473]
	GetHandleVerifier [0x012695F2+624802]
	GetHandleVerifier [0x01270E6C+655644]
	(No symbol) [0x01172C9D]
	(No symbol) [0x0116FD68]
	(No symbol) [0x0116FF05]
	(No symbol) [0x01162336]
	BaseThreadInitThunk [0x76507BA9+25]
	RtlInitializeExceptionChain [0x77B1C11B+107]
	RtlClearBits [0x77B1C09F+191]


In [213]:
driver.switch_to.frame("mainFrame") #MAIN을 바꾸자

In [215]:
driver.find_element(By.CSS_SELECTOR, "div.pcol1").text 
#필요한 속성이 들어있기만하면 div.pcol1을 들고오든 p.se-text-paragraph se-text-paragraph-align-를 가져오든 상관없음
# 상위 속성은 div.pcol1을 들고온 이유는 짧아서일뿐

'발망 바이 아코니(BALMAIN by Akoni) - LEGION ll BLK-GLD By 안경 쓴 거북이.'

## 주가사이트에서 스크랩핑

F12에서 network 선택하면 네트워크 반응을 확인할 수 있음\
그 후 하단 Name의 제일 처음 선택지의 오른쪽 payload를 보면 뭐가 선택되어 적용된건지 확인 가능

In [262]:
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import time
import pandas as pd

In [270]:
url = requests.get('https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page=1')
html = BeautifulSoup(url.text)

- 가끔 속성에 class가 없는 경우가 있음\
  'name'을 가져오기

In [273]:
fields = []
for i in html.find_all("input", attrs = {"name" : "fieldIds"}):
    fields.append(i['value'])
print(fields)

['quant', 'ask_buy', 'amount', 'market_sum', 'operating_profit', 'per', 'open_val', 'ask_sell', 'prev_quant', 'property_total', 'operating_profit_increasing_rate', 'roe', 'high_val', 'buy_total', 'frgn_rate', 'debt_total', 'net_income', 'roa', 'low_val', 'sell_total', 'listed_stock_cnt', 'sales', 'eps', 'pbr', 'sales_increasing_rate', 'dividend', 'reserve_ratio']


In [283]:
total = []

for n in tqdm(range(1, 47)):

    dic = {"menu" : 'market_sum',
           "returnUrl" : f"http://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page={n}",
           "fieldIds" : fields}

    url = requests.post("https://finance.naver.com/sise/field_submit.naver", data = dic)
    html = BeautifulSoup(url.text)

    table = html.find("table", class_ = 'type_2')
    table = pd.read_html(str(table))[0]
    total.append(table)
    time.sleep(2)

100%|██████████| 46/46 [01:47<00:00,  2.34s/it]


In [285]:
pd.concat(total)

,N,종목명,현재가,전일비,등락률,액면가,거래량,거래대금,전일거래량,시가,...,보통주배당금,매출액증가율,영업이익증가율,외국인비율,PER,ROE,ROA,PBR,유보율,토론실
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,삼성전자,60800.0,상승 200,+0.33%,100.0,32607072.0,1968615.0,24247578.0,60200.0,...,1444.0,-14.33,-84.86,53.68,14.86,4.15,3.43,1.11,39114.3,NaN
2,2.0,SK하이닉스,183300.0,"상승 9,200",+5.28%,5000.0,3475958.0,626588.0,3633358.0,178400.0,...,1200.0,-26.57,-213.52,54.00,53.41,-15.61,-8.95,2.11,1397.1,NaN
3,3.0,LG에너지솔루션,420500.0,"상승 17,000",+4.21%,500.0,418702.0,175929.0,255155.0,413500.0,...,NaN,31.83,78.23,4.96,-585.65,6.36,3.91,4.77,16691.6,NaN
4,4.0,삼성바이오로직스,998000.0,"하락 1,000",-0.10%,2500.0,62492.0,62357.0,138681.0,1006000.0,...,NaN,23.10,13.22,13.33,69.07,9.12,5.26,6.88,5432.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,2257.0,KOSEF 차이나A50커넥트인버스MSCI(합성 H),8445.0,하락 30,-0.35%,0.0,153.0,1.0,1933.0,8445.0,...,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN
11,2258.0,KOSEF Fn중소형,20435.0,상승 145,+0.71%,0.0,262.0,5.0,55.0,20315.0,...,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
